# Aprendizado de Máquina - TP1
# Classificação de Exoplanetas

O trabalho tem como objetivo a prática de conceitos aprendidos na disciplina, adquirindo experiência no uso de métodos de classificação, na avaliação de modelos, e na interpretação e apresentação de resultados de experimentos. Para isso, utilizaremos e compararemos diferentes métodos para solucionar um problema de classificação binária.  

O problema abordado é o da classificação de exoplanetas identificados pela sonda espacial *Kepler* entre **confirmados** e **falsos positivos**. Os possíveis exoplanetas são chamados **Kepler Object of Interest (KOI)**, e cada observação do conjunto de dados corresponde a um KOI e suas características estimadas.

Serão explorados os seguintes métodos de classificação:
- [ ] **Naive Bayes**
    - [ ] Apenas um experimento, para servir de baseline.
- [ ] **Decision Tree**
    - [ ] Variação de altura máxima da árvore, incluindo ilimitada
    - [ ] Visualização gráfica dos resultados
- [ ] **SVM**
    - [ ] Avaliação dos Kernels
        - [ ] Linear
        - [ ] Sigmoid
        - [ ] Polinomial
        - [ ] RBF
- [ ] **k-NN**
    - [ ] Variação do número de vizinhos *k*
    - [ ] Visualização gráfica dos resultados
- [ ] **Random Forest**
    - [ ] Variação do número de árvores
    - [ ] Visualização gráfica dos resultados
- [ ] **Gradient Tree Boosting**
    - [ ] Variação do número de iterações
    - [ ] Visualização gráfica dos resultados

Os métodos estão disponíveis no módulo `scikit-learn`. Iremos utilizar também os módulos `numpy`, `matplotlib.pyplot` e `pandas` para operações matemáticas, geração de gráficos, e manipulação do conjunto de dados, respectivamente.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Definimos um _seed_ para que os resultados do _notebook_ sejam reproduzíveis:

In [18]:
seed = 7

In [2]:
koi = pd.read_csv("koi_data.csv")
koi.isnull().sum()

kepoi_name          0
koi_disposition     0
koi_period          0
koi_impact          0
koi_duration        0
koi_depth           0
koi_ror             0
koi_srho            0
koi_prad            0
koi_sma             0
koi_incl            0
koi_teq             0
koi_insol           0
koi_dor             0
koi_max_sngle_ev    0
koi_max_mult_ev     0
koi_model_snr       0
koi_steff           0
koi_slogg           0
koi_smet            0
koi_srad            0
koi_smass           0
koi_kepmag          0
koi_gmag            0
koi_rmag            0
koi_imag            0
koi_zmag            0
koi_jmag            0
koi_hmag            0
koi_kmag            0
koi_fwm_stat_sig    0
koi_fwm_sra         0
koi_fwm_sdec        0
koi_fwm_srao        0
koi_fwm_sdeco       0
koi_fwm_prao        0
koi_fwm_pdeco       0
koi_dicco_mra       0
koi_dicco_mdec      0
koi_dicco_msky      0
koi_dikco_mra       0
koi_dikco_mdec      0
koi_dikco_msky      0
dtype: int64

Não há dados faltantes no conjunto de dados.

In [5]:
target = koi["koi_disposition"] # Classificação de interesse
koi = koi.drop("koi_disposition", axis=1) # Para usar os métodos do scikit-learn é cômodo separar o atributo alvo
target.describe()

count               5202
unique                 2
top       FALSE POSITIVE
freq                3098
Name: koi_disposition, dtype: object

Temos duas classes de interesse, e pela proporção observada de falsos positivos (3098/5202), não há sub-representação.

---

In [23]:
koi.describe()

,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
count,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,...,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000
mean,37.032237,0.717106,5.607025,21340.318993,0.235205,3.415370,112.230798,0.158146,81.181413,1143.721069,...,-0.355681,-0.805629,-0.000263,0.000439,-0.049743,-0.087413,1.930251,-0.038402,-0.098738,1.920226
std,88.417985,2.628207,6.962634,66989.808550,2.586213,25.131368,3699.799318,0.241792,16.308839,775.788868,...,10.978677,14.741473,0.065707,0.077519,2.465670,2.746534,3.147553,2.465094,2.734732,3.142764
min,0.306940,0.000000,0.104600,0.800000,0.001290,0.000040,0.080000,0.007200,2.290000,92.000000,...,-275.600000,-397.620000,-4.000000,-0.800000,-21.500000,-75.900000,0.000000,-23.600000,-76.600000,0.000000
25%,2.213962,0.226000,2.500250,176.800000,0.013058,0.176092,1.460000,0.033000,81.930000,615.250000,...,-0.500000,-0.570000,-0.000240,-0.000240,-0.270000,-0.291500,0.128250,-0.265250,-0.320000,0.180000
50%,7.386755,0.610000,3.805500,495.950000,0.024185,0.748045,2.600000,0.073650,87.890000,948.000000,...,0.000000,-0.030000,0.000000,0.000000,0.000000,0.000000,0.460000,-0.007000,-0.018000,0.453000
75%,23.448117,0.923750,6.000750,2120.525000,0.171260,2.267063,21.645000,0.158200,89.520000,1482.000000,...,0.500000,0.450000,0.000260,0.000280,0.230000,0.230000,2.570000,0.226250,0.250000,2.420000
max,1071.232620,100.806000,138.540000,864260.000000,99.870650,918.752390,200346.000000,2.034500,90.000000,9791.000000,...,97.780000,98.780000,1.190000,5.000000,45.680000,27.500000,88.600000,46.570000,31.200000,89.600000


**Temos muita variação entre as grandezas dos atributos.** (e.g: `koi_depth` e `koi_fwm_pdeco`)  
**Deve ser interessante normalizar os dados para aplicação de certos modelos.**

# Validação Cruzada K-Fold

Para evitar [*overfitting*](https://en.wikipedia.org/wiki/Overfitting) em um classificador, dividimos o conjunto de dados em subconjuntos de **treino** e **teste**. Essa técnica é chamada de [**validação cruzada**.](https://en.wikipedia.org/wiki/Cross-validation_(statistics))
- **Treino**: dados a partir dos quais serão estimados os parâmetros do classificador.
- **Teste**: dados usados para avaliar a acurácia do classificador para dados desconhecidos (i.e., avaliar sua generalização).

Existem diversas estratégias de validação cruzada. Neste trabalho usaremos a validação **k-fold**, a fim de estimar a qualidade de um modelo com boa confiabilidade. Ela se dá pelos seguintes passos:
- O conjunto de dados é dividido em *k* partes de tamanhos iguais;
- O modelo é treinado _k_ vezes, em cada uma delas usando um dos subconjuntos como conjunto de teste e a união dos *k-1* subconjuntos restantes como conjunto de treinamento;
    - Para cada iteração, o modelo é avaliado.
- Por fim, a qualidade do modelo é sumarizada a partir dos resultados obtidos.

Para tanto, aproveitaremos a implementação `KFold`, presente no módulo `sklearn`. Se uma das classes de exoplanetas estivesse sub-representada, seria interessante formar subconjuntos estratificados com o `StratifiedKFold`.

In [21]:
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle=True, random_state=seed) # 5 e 10 são valores comumente escolhidos para k

Exemplo de funcionamento do `KFold`:

In [22]:
for train, test in kfold.split(koi[:50], target[:50]):
    print("TRAIN:", train, "TEST:", test)

TRAIN: [ 0  2  3  4  5  6  7  8  9 11 12 14 16 17 18 19 21 23 24 25 26 28 29 31
 32 33 34 35 37 38 39 40 41 43 44 45 46 47 48 49] TEST: [ 1 10 13 15 20 22 27 30 36 42]
TRAIN: [ 0  1  3  4  5  6  7  8 10 11 12 13 14 15 16 19 20 21 22 23 24 25 26 27
 28 30 31 33 36 37 38 39 40 42 43 44 45 47 48 49] TEST: [ 2  9 17 18 29 32 34 35 41 46]
TRAIN: [ 0  1  2  3  4  7  8  9 10 11 13 14 15 17 18 19 20 22 23 25 26 27 28 29
 30 32 34 35 36 37 39 40 41 42 43 44 46 47 48 49] TEST: [ 5  6 12 16 21 24 31 33 38 45]
TRAIN: [ 1  2  3  4  5  6  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 25 27 28
 29 30 31 32 33 34 35 36 38 39 41 42 44 45 46 47] TEST: [ 0  7  8 11 26 37 40 43 48 49]
TRAIN: [ 0  1  2  5  6  7  8  9 10 11 12 13 15 16 17 18 20 21 22 24 26 27 29 30
 31 32 33 34 35 36 37 38 40 41 42 43 45 46 48 49] TEST: [ 3  4 14 19 23 25 28 39 44 47]


# Naive Bayes

Faremos apenas um experimento, que servirá como uma primeira tentativa de solução do problema, e base de comparação entre outros métodos.